***
    Copyright 2020, by the California Institute of Technology. ALL RIGHTS RESERVED. 
    United States Government Sponsorship acknowledged. Any commercial use must be negotiated 
    with the Office of Technology Transfer at the California Institute of Technology.
    
    This software may be subject to U.S. export control laws. By accepting this software, 
    the user agrees to comply with all applicable U.S. export laws and regulations. 
    User has the responsibility to obtain export licenses, or other export authority as may be 
    required before exporting such information to foreign countries or providing access to foreign persons.
***



# Tumor/Stroma classification using Random Forests

Cells are classified into *tumor* or *stroma* classes based on antigen expression levels of each cell.  The following antigens are available:

`['CCR3', 'CD103', 'CD11B', 'CD141', 'CD163', 'CD19', 'CD3', 'CD31', 'CD4', 'CD45RA', 'CD56', 'CD62L', 'CD8', 'CXCR3', 'EPCAM', 'FAP', 'FOXP3', 'GFAP', 'GZMB', 'IL10', 'INOS', 'KI67', 'LOX1', 'TRYPTASE', 'MPO', 'MUC1', 'PD1', 'PDL1', 'PRG2', 'SMA', 'VIMENTIN']`

We sample from a total of 268074 cells from 6 patients.  Upon fitting the models, antigens are then ranked with feature importance.  The scripts are designed to test the following tests for `all patients` and `individual patients`:

* `all antigens`
* `all antigens but MUC1, KI67, EPCAM`
* `all but top K antigens by intensity means`
* `only top K antigens found after ranking all antigens`







In [1]:
import pandas as pd 
import numpy as np 
import glob2
import os

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn import metrics

import glob2
import json

import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
# input and output folders
indir = 'data/'
outdir = 'out/'

***
# Optimizing RF parameters
***

In [3]:
files = glob2.glob(os.path.join(indir,'*.csv'))
d = (pd.read_csv(f) for f in files)
df = pd.concat(d, ignore_index=True)
print(len(df),' Stroma:',len(df[df['TARGET']=='stroma']), ' Tumor:',len(df[df['TARGET']=='tumor']))

df['TARGET'] = (df['TARGET'] == 'stroma').astype('int')

x = df.drop('TARGET', axis=1)
y = df['TARGET']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

rf = RandomForestClassifier(class_weight='balanced_subsample', random_state=42)
rf.fit(x_train, y_train)
y_pred = rf.predict(x_test)

print(json.dumps(rf.get_params(), indent=2))
random_grid = {'n_estimators': [10,20,50,90,100,120,150],
                'max_features': ['auto', 'sqrt', "log2"],
                'max_depth': [10, 20, 30, 40, 50, 60], 
                'min_samples_split': [2, 5, 10],
                'min_samples_leaf': [1, 2, 4, 8], 
                'bootstrap': [True, False]}

# random search the param grid
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,
                n_iter = 50, scoring = 'f1', verbose = 2,
                cv = 5, random_state = 42, n_jobs = -1, return_train_score = True)

rf_random.fit(x_train, y_train)
print('Best params:', json.dumps(rf_random.best_params_, indent=2))
print('Best score:', rf_random.best_score_)
print('Score on test:', rf_random.score(x_test,y_test))

# grid search the neighborhood of random search
nest = rf_random.best_params_['n_estimators']
maxd = rf_random.best_params_['max_depth']
param_grid = {'n_estimators': list(range(nest-3,nest+4)), 
                'max_depth': list(range(maxd-3,maxd+4))}
rf = RandomForestClassifier(random_state = 42)
rf_grid = GridSearchCV(estimator = rf, param_grid = param_grid, scoring = 'f1', 
                cv = 5, verbose = 2, n_jobs = -1, return_train_score = True)

rf_grid.fit(x_train, y_train)
print('Best grid params:', json.dumps(rf_grid.best_params_, indent=2))
print('Best score:', rf_random.best_score_)
print('Score on test:', rf_grid.score(x_test,y_test))

268074  Stroma: 93827  Tumor: 174247
{
  "bootstrap": true,
  "ccp_alpha": 0.0,
  "class_weight": "balanced_subsample",
  "criterion": "gini",
  "max_depth": null,
  "max_features": "auto",
  "max_leaf_nodes": null,
  "max_samples": null,
  "min_impurity_decrease": 0.0,
  "min_impurity_split": null,
  "min_samples_leaf": 1,
  "min_samples_split": 2,
  "min_weight_fraction_leaf": 0.0,
  "n_estimators": 100,
  "n_jobs": null,
  "oob_score": false,
  "random_state": 42,
  "verbose": 0,
  "warm_start": false
}
Fitting 5 folds for each of 50 candidates, totalling 250 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
/Applications/anaconda3/envs/unmc_rdf/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 250 out of 250 | elapsed: 18.8min finished


Best params: {
  "n_estimators": 150,
  "min_samples_split": 10,
  "min_samples_leaf": 8,
  "max_features": "sqrt",
  "max_depth": 30,
  "bootstrap": false
}
Best score: 0.840244671641523
Score on test: 0.8450809278734989
Fitting 5 folds for each of 49 candidates, totalling 245 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 32 concurrent workers.
/Applications/anaconda3/envs/unmc_rdf/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py:688: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(
[Parallel(n_jobs=-1)]: Done  98 tasks      | elapsed: 15.9min
[Parallel(n_jobs=-1)]: Done 245 out of 245 | elapsed: 31.3min finished


Best grid params: {
  "max_depth": 32,
  "n_estimators": 153
}
Best score: 0.840244671641523
Score on test: 0.8004191752111288


***
# All patients
***

In [4]:
def go(x, y, outtxt, outfig, ct, k):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
    rdf = RandomForestClassifier(n_estimators=rf_grid.best_params_['n_estimators'],
                                max_depth=rf_grid.best_params_['max_depth'],
                                class_weight='balanced_subsample')
    rdf.fit(x_train, y_train)
    y_pred = rdf.predict(x_test)

    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
    scores = cross_val_score(rdf, x, y, scoring='accuracy', cv=cv, n_jobs=-1)

    feature_imp = pd.Series(rdf.feature_importances_, index=x_train.columns).sort_values(ascending=False)

    with open(outtxt, 'a') as out:
        out.write('Prediction accuracy: {:6.2%}\n\n'.format( metrics.accuracy_score(y_test, y_pred) ))      
        out.write('Scores: {}\n\n'.format(scores))
        out.write('Mean accuracy: {:6.2%}\n\n'.format(np.mean(scores)))
        out.write('Antigen importances:\n\n')
        out.write(feature_imp.to_string()+'\n\n')

    plt.figure(figsize=(9,7))
    sns.barplot(x=feature_imp, y=feature_imp.index, palette='flare')

    plt.xlabel('Importance')
    plt.ylabel('Antigens')
    plt.title(ct)
    plt.savefig(outfig, dpi=300)
    plt.close()

    # redo with only top K antigens found above
    x.drop(x.columns.difference(list(feature_imp[:K].index)), axis=1, inplace=True)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

    rdf.fit(x_train, y_train)
    y_pred = rdf.predict(x_test)

    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
    scores = cross_val_score(rdf, x, y, scoring='accuracy', cv=cv, n_jobs=-1)

    feature_imp = pd.Series(rdf.feature_importances_, index=x_train.columns).sort_values(ascending=False)

    with open(outtxt, 'a') as out:
        out.write('Top '+str(k)+' antigens found in previous run:\n\n')
        out.write('Prediction accuracy: {:6.2%}\n\n'.format( metrics.accuracy_score(y_test, y_pred) ))      
        out.write('Scores: {}\n\n'.format(scores))
        out.write('Mean accuracy: {:6.2%}\n\n'.format(np.mean(scores)))
        out.write('Antigen importances:\n\n')
        out.write(feature_imp.to_string()+'\n\n')

    plt.figure(figsize=(9,7))
    sns.barplot(x=feature_imp, y=feature_imp.index, palette='flare')

    plt.xlabel('Importance')
    plt.ylabel('Antigens')
    plt.title(ct+' top '+str(k))
    plt.savefig(outfig.replace('.pdf','_top_'+str(k)+'.pdf'), dpi=300)
    plt.close()


# all patients 
files = glob2.glob(os.path.join(indir,'*.csv'))
d = (pd.read_csv(f) for f in files)
df = pd.concat(d, ignore_index=True)

K = 3

# all antigens
x_all = df.drop('TARGET', axis=1)
y_all = df['TARGET']

out_all_txt = os.path.join(outdir, 'all_patients_all_antigens.txt')
out_all_fig = os.path.join(outdir, 'all_patients_all_antigens.pdf')
chart_title = 'All patients - all antigens'

with open(out_all_txt, 'a') as o:
    o.write('Stroma: {} cells\n'.format(len( df[df['TARGET']=='stroma'] )))
    o.write('Tumor: {} cells\n\n'.format(len( df[df['TARGET']=='tumor'] )))
    o.write('Antigen means: \n\n{}\n\n'.format(df.mean().sort_values(ascending=False).to_string()))
    o.write('Antigen min: \n\n{}\n\n'.format(df.min().to_string()))
    o.write('Antigen max: \n\n{}\n\n'.format(df.max().to_string()))
    
go(x_all, y_all, out_all_txt, out_all_fig, chart_title, K)

# all but MUC1, KI67, EPCAM
dmke = df.drop(['MUC1','KI67','EPCAM'], axis=1)
x_mke = dmke.drop('TARGET', axis=1)
y_mke = dmke['TARGET']

with open(out_all_txt, 'a') as o:
    o.write('Running with no MUC1, KI67, EPCAM:\n\n')

out_all_fig = os.path.join(outdir, 'all_patients_no_muc1_ki67_epcam.pdf')
chart_title = 'All patients - no MUC1, KI67, EPCAM'
go(x_mke, y_mke, out_all_txt, out_all_fig, chart_title, K)

# exclude top K antigens by mean expression level
ds = df.mean().sort_values(ascending=False)
exclude = ds.index[:K]
df = df.drop(exclude, axis=1)

x_excl = df.drop('TARGET', axis=1)
y_excl = df['TARGET']

with open(out_all_txt, 'a') as o:
    o.write('Running after excluding top '+str(K)+' antigens by mean intensity:\n\n')

out_all_fig = os.path.join(outdir, 'all_patients_skip_top_'+str(K)+'.pdf')
chart_title = 'All patients - skip top '+str(K)+' by mean intensity'
go(x_excl, y_excl, out_all_txt, out_all_fig, chart_title, K)


***
# Individual patients
***

In [7]:
files = glob2.glob(os.path.join(indir,'*.csv'))

K = 5

for f in files:
    fm = os.path.splitext(os.path.basename(f))[0]
    df = pd.read_csv(f)

    # all antigens
    x_all = df.drop('TARGET', axis=1)
    y_all = df['TARGET']

    out_all_txt = os.path.join(outdir, 'patient_'+fm+'_all_antigens.txt')
    out_all_fig = os.path.join(outdir, 'patient_'+fm+'_all_antigens.pdf')
    chart_title = 'Patient '+fm+' - all antigens'

    with open(out_all_txt, 'a') as o:
        o.write('Stroma: {} cells\n'.format(len( df[df['TARGET']=='stroma'] )))
        o.write('Tumor: {} cells\n\n'.format(len( df[df['TARGET']=='tumor'] )))
        o.write('Antigen means: \n\n{}\n\n'.format(df.mean().sort_values(ascending=False).to_string()))
        o.write('Antigen min: \n\n{}\n\n'.format(df.min().to_string()))
        o.write('Antigen max: \n\n{}\n\n'.format(df.max().to_string()))
    
    go(x_all, y_all, out_all_txt, out_all_fig, chart_title, K)

    # all but MUC1, KI67, EPCAM
    dmke = df.drop(['MUC1','KI67','EPCAM'], axis=1)
    x_mke = dmke.drop('TARGET', axis=1)
    y_mke = dmke['TARGET']

    with open(out_all_txt, 'a') as o:
        o.write('Running with no MUC1, KI67, EPCAM:\n\n')

    out_all_fig = os.path.join(outdir, 'patient_'+fm+'_no_muc1_ki67_epcam.pdf')
    chart_title = 'Patient '+fm+' - no MUC1, KI67, EPCAM'
    go(x_mke, y_mke, out_all_txt, out_all_fig, chart_title, K)

    # exclude top K antigens by mean expression level
    ds = df.mean().sort_values(ascending=False)
    exclude = ds.index[:K]
    df = df.drop(exclude, axis=1)

    x_excl = df.drop('TARGET', axis=1)
    y_excl = df['TARGET']

    with open(out_all_txt, 'a') as o:
        o.write('Running after excluding top '+str(K)+' antigens by mean intensity:\n\n')

    out_all_fig = os.path.join(outdir, 'patient_'+fm+'_skip_top_'+str(K)+'.pdf')
    chart_title = 'Patient '+fm+' - skip top '+str(K)+' by mean intensity'
    go(x_excl, y_excl, out_all_txt, out_all_fig, chart_title, K)

***
# Leave one out

## All patients all antigens, hold out one patient at a time
## Test model on held out patient
***

In [11]:
def jk(x, y, outtxt, outfig, ct, k):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)
    rdf = RandomForestClassifier(n_estimators=rf_grid.best_params_['n_estimators'],
                                max_depth=rf_grid.best_params_['max_depth'],
                                class_weight='balanced_subsample')
    rdf.fit(x_train, y_train)
    y_pred = rdf.predict(x_test)

    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
    scores = cross_val_score(rdf, x, y, scoring='accuracy', cv=cv, n_jobs=-1)

    feature_imp = pd.Series(rdf.feature_importances_, index=x_train.columns).sort_values(ascending=False)

    with open(outtxt, 'a') as out:
        out.write('Prediction accuracy: {:6.2%}\n\n'.format( metrics.accuracy_score(y_test, y_pred) ))      
        out.write('Scores: {}\n\n'.format(scores))
        out.write('Mean accuracy: {:6.2%}\n\n'.format(np.mean(scores)))
        out.write('Antigen importances:\n\n')
        out.write(feature_imp.to_string()+'\n\n')

    plt.figure(figsize=(9,7))
    sns.barplot(x=feature_imp, y=feature_imp.index, palette='flare')

    plt.xlabel('Importance')
    plt.ylabel('Antigens')
    plt.title(ct)
    plt.savefig(outfig, dpi=300)
    plt.close()

    # redo with only top K antigens found above
    exclude = list(feature_imp[:K].index)
    x.drop(x.columns.difference(exclude), axis=1, inplace=True)
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

    rdf_k = RandomForestClassifier(n_estimators=rf_grid.best_params_['n_estimators'],
                                max_depth=rf_grid.best_params_['max_depth'],
                                class_weight='balanced_subsample')
    rdf_k.fit(x_train, y_train)
    y_pred = rdf_k.predict(x_test)

    cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=1)
    scores = cross_val_score(rdf_k, x, y, scoring='accuracy', cv=cv, n_jobs=-1)

    feature_imp = pd.Series(rdf_k.feature_importances_, index=x_train.columns).sort_values(ascending=False)

    with open(outtxt, 'a') as out:
        out.write('Top '+str(k)+' antigens found in previous run:\n\n')
        out.write('Prediction accuracy: {:6.2%}\n\n'.format( metrics.accuracy_score(y_test, y_pred) ))      
        out.write('Scores: {}\n\n'.format(scores))
        out.write('Mean accuracy: {:6.2%}\n\n'.format(np.mean(scores)))
        out.write('Antigen importances:\n\n')
        out.write(feature_imp.to_string()+'\n\n')

    plt.figure(figsize=(9,7))
    sns.barplot(x=feature_imp, y=feature_imp.index, palette='flare')

    plt.xlabel('Importance')
    plt.ylabel('Antigens')
    plt.title(ct+' top '+str(k))
    plt.savefig(outfig.replace('.pdf','_top_'+str(k)+'.pdf'), dpi=300)
    plt.close()
    return rdf, rdf_k, exclude




files = glob2.glob(os.path.join(indir,'*.csv'))

K = 5

for i,f in enumerate(files):
    training_files = files[:i] + files[i+1:]
    testing_file = files[i]

    print('Training:')
    print(training_files)
    print('Held out:')
    print(testing_file)

    d = (pd.read_csv(f) for f in training_files)
    df = pd.concat(d, ignore_index=True)

    fm = os.path.splitext(os.path.basename(testing_file))[0]

    # all antigens
    x_all = df.drop('TARGET', axis=1)
    y_all = df['TARGET']

    out_all_txt = os.path.join(outdir, 'all_patients_but_'+fm+'_all_antigens.txt')
    out_all_fig = os.path.join(outdir, 'all_patients_but_'+fm+'_all_antigens.pdf')
    chart_title = 'All patients but '+fm+' - all antigens'

    with open(out_all_txt, 'a') as o:
        o.write('Stroma: {} cells\n'.format(len( df[df['TARGET']=='stroma'] )))
        o.write('Tumor: {} cells\n\n'.format(len( df[df['TARGET']=='tumor'] )))
        o.write('Antigen means: \n\n{}\n\n'.format(df.mean().sort_values(ascending=False).to_string()))
        o.write('Antigen min: \n\n{}\n\n'.format(df.min().to_string()))
        o.write('Antigen max: \n\n{}\n\n'.format(df.max().to_string()))
    
    rf_all, rf_top, excluded = jk(x_all, y_all, out_all_txt, out_all_fig, chart_title, K)

    # held out patient
    tf = pd.read_csv(testing_file)

    # all antigens
    x = tf.drop('TARGET', axis=1)
    y = tf['TARGET']
    y_pred_all = rf_all.predict(x)
    with open(out_all_txt, 'a') as out:
        out.write('Prediction accuracy all antigens: {:6.2%}\n\n'.format( metrics.accuracy_score(y, y_pred_all) ))    

    x.drop(x.columns.difference(excluded), axis=1, inplace=True)
    y_pred_top = rf_top.predict(x)
    with open(out_all_txt, 'a') as out:
        out.write('Prediction accuracy top antigens: {:6.2%}\n\n'.format( metrics.accuracy_score(y, y_pred_top) ))    

    # all but MUC1, KI67, EPCAM
    dmke = df.drop(['MUC1','KI67','EPCAM'], axis=1)
    x = dmke.drop('TARGET', axis=1)
    y = dmke['TARGET']

    with open(out_all_txt, 'a') as o:
        o.write('Running with no MUC1, KI67, EPCAM:\n\n')

    out_all_fig = os.path.join(outdir, 'all_patients_but_'+fm+'_no_muc1_ki67_epcam.pdf')
    chart_title = 'All patients but '+fm+' - no MUC1, KI67, EPCAM'
    rf_all, rf_top, excluded = jk(x, y, out_all_txt, out_all_fig, chart_title, K)

    dmke = tf.drop(['MUC1','KI67','EPCAM'], axis=1)
    x = dmke.drop(['TARGET'], axis=1)
    y = dmke['TARGET']

    y_pred_all = rf_all.predict(x)
    with open(out_all_txt, 'a') as out:
        out.write('Prediction accuracy all antigens but MUC1, KI67, EPCAM: {:6.2%}\n\n'.format( metrics.accuracy_score(y, y_pred_all) ))    

    x.drop(x.columns.difference(excluded), axis=1, inplace=True)
    y_pred_top = rf_top.predict(x)
    with open(out_all_txt, 'a') as out:
        out.write('Prediction accuracy top antigens but MUC1, KI67, EPCAM: {:6.2%}\n\n'.format( metrics.accuracy_score(y, y_pred_top) ))

  


Training:
['data/86.csv', 'data/116.csv', 'data/107.csv', 'data/89.csv', 'data/8.csv']
Held out:
data/80.csv
Training:
['data/80.csv', 'data/116.csv', 'data/107.csv', 'data/89.csv', 'data/8.csv']
Held out:
data/86.csv
Training:
['data/80.csv', 'data/86.csv', 'data/107.csv', 'data/89.csv', 'data/8.csv']
Held out:
data/116.csv
Training:
['data/80.csv', 'data/86.csv', 'data/116.csv', 'data/89.csv', 'data/8.csv']
Held out:
data/107.csv
Training:
['data/80.csv', 'data/86.csv', 'data/116.csv', 'data/107.csv', 'data/8.csv']
Held out:
data/89.csv
Training:
['data/80.csv', 'data/86.csv', 'data/116.csv', 'data/107.csv', 'data/89.csv']
Held out:
data/8.csv


***
# Tree plotting

https://scikit-learn.org/stable/modules/generated/sklearn.tree.plot_tree.html
***

# OOB scoring
https://scikit-learn.org/stable/auto_examples/ensemble/plot_ensemble_oob.html
***